# Group 7
# Big Data Group Project

## Contributors:
- 29233798: Joel Shien Yee Chin [Joel]
- 18435688: Tim O'Doherty [Tim]
- 22606127: Sean Whitehead [Sean]
- 29650437: Lin Bai [Lin]


## Phase 3: Machine Learning Pipeline

## 3.1 Load Libraries

In [1]:
import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}

Intitializing Scala interpreter ...

Spark Web UI available at http://b335dbce4d27:4040
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1560072779294)
SparkSession available as 'spark'


2019-06-09 09:32:55,706 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}


## 3.2 Data loading

In [2]:
// Load wrangled datasets from Phase 2: part1.csv; part2.csv; part3.csv

val creditRiskdf1 = spark.read.option("header","true").csv("part1.csv")
val creditRiskdf2 = spark.read.option("header","true").csv("part2.csv")
val creditRiskdf3 = spark.read.option("header","true").csv("part3.csv")

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [3]:
// combine the 3 dataframes to 1
val creditRiskdf1and2 = creditRiskdf1.union(creditRiskdf2)
val creditRiskdf = creditRiskdf1and2.union(creditRiskdf3)

creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [4]:
creditRiskdf.cache()

res0: creditRiskdf.type = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [5]:
// check that the rows have been aggregated correctly

println("Total data rows in loaded dataframes:"+(
    creditRiskdf1.count()+creditRiskdf2.count()+creditRiskdf3.count()))

println("Total data rows in combined dataframe:"+creditRiskdf.count())

2019-06-09 09:33:50,305 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
Total data rows in loaded dataframes:183875
Total data rows in combined dataframe:183875


**Output**
- creditRiskdf: loaded and consolidated dataset.

## 3.3 Data cleansing and reshaping

In [6]:
// Drop features with inconsistent values from dataset.

val incon_droplist = List("previous_loans_DAYS_FIRST_DUE_mean",
                      "previous_loans_SELLERPLACE_AREA_min",
                      "previous_loans_DAYS_FIRST_DUE_sum")

val df = creditRiskdf.drop(incon_droplist:_*)

incon_droplist: List[String] = List(previous_loans_DAYS_FIRST_DUE_mean, previous_loans_SELLERPLACE_AREA_min, previous_loans_DAYS_FIRST_DUE_sum)
df: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [7]:
var creditRiskdf = df

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [8]:
// change datatype of data from String to Float
var creditRiskFeatures = creditRiskdf.columns

for (colName<-creditRiskFeatures){
     |   creditRiskdf = creditRiskdf.withColumn(
         colName,col(colName).cast("Float"))
     | }

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CR...

In [9]:
creditRiskdf.printSchema()

root
 |-- EXT_SOURCE_2: float (nullable = true)
 |-- EXT_SOURCE_3: float (nullable = true)
 |-- client_installments_AMT_PAYMENT_min_sum: float (nullable = true)
 |-- DAYS_BIRTH: float (nullable = true)
 |-- AMT_CREDIT: float (nullable = true)
 |-- AMT_ANNUITY: float (nullable = true)
 |-- DAYS_EMPLOYED: float (nullable = true)
 |-- bureau_DAYS_CREDIT_ENDDATE_max: float (nullable = true)
 |-- bureau_DAYS_CREDIT_max: float (nullable = true)
 |-- DAYS_ID_PUBLISH: float (nullable = true)
 |-- bureau_DAYS_ENDDATE_FACT_max: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_DEBT_mean: float (nullable = true)
 |-- previous_loans_CNT_PAYMENT_mean: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_FUTURE_min_max: float (nullable = true)
 |-- previous_loans_SELLERPLACE_AREA_max: float (nullable = true)
 |-- DAYS_LAST_PHONE_CHANGE: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_max: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_mean: float (nullable = true)
 |-- DAYS_REGISTRATION: 

 |-- previous_loans_NAME_GOODS_CATEGORY_Mobile_count_norm: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_OVERDUE_mean: float (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Construction_count_norm: float (nullable = true)
 |-- previous_loans_CODE_REJECT_REASON_SCOFR_count_norm: float (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Furniture_count_norm: float (nullable = true)
 |-- client_cash_NAME_CONTRACT_STATUS_Active_count_norm_max: float (nullable = true)
 |-- previous_loans_NAME_GOODS_CATEGORY_Construction Materials_count_norm: float (nullable = true)
 |-- client_bureau_balance_STATUS_1_count_norm_sum: float (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_Cash Street: middle_count_norm: float (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_POS other with interest_count_norm: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_min_min: float (nullable = true)
 |-- previous_loans_CODE_REJECT_REASON_VERIF_count_norm: float (nullable 

In [10]:
//string array of features without label "TARGET"

creditRiskFeatures = creditRiskFeatures.filter(! _.contains("TARGET"))
creditRiskFeatures.length

creditRiskFeatures: Array[String] = [Ljava.lang.String;@5a2fab75
res4: Int = 281


In [11]:
// split features array to facilitate processing. Exclude feature SK_ID.

val creditRiskFeatures1 = creditRiskFeatures.slice(0,39)
val creditRiskFeatures2 = creditRiskFeatures.slice(40,79)
val creditRiskFeatures3 = creditRiskFeatures.slice(80,119)
val creditRiskFeatures4 = creditRiskFeatures.slice(120,159)
val creditRiskFeatures5 = creditRiskFeatures.slice(160,199)
val creditRiskFeatures6 = creditRiskFeatures.slice(200,239)
val creditRiskFeatures7 = creditRiskFeatures.slice(240,280)

creditRiskFeatures1: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_C...

In [12]:
// check number of data lines in dataframe creditRiskdf

val rows = creditRiskdf.select($"TARGET").count()

rows: Long = 183875


In [13]:
// set up global variables

var creditRiskdf1 = creditRiskdf

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 280 more fields]


### Run above cells up to this block if directly loading highOutlier and lowOutlier lists from text files.

In [14]:
//write function to check if there are any outliers in a given domain and show boxplot [Sean]
//slice a small test list to ensure functionality
//var test_lis = creditRiskFeatures.slice(0,2) //TEST var for codeblock

var highOutlier_list = List[String]() //empty list to append features containing outliers in
var lowOutlier_list = List[String]() //empty list to append features containing outliers in
var feature_dropThreshold = 0.05*rows
var row_dropLower = 0.02*rows

def outliers(feature_set: Array[String],creditRiskdf:org.apache.spark.sql.DataFrame){    
    for (feature <- feature_set) {
        //summarise column and query the DataFrame output
        var firstQ = creditRiskdf.select(feature).summary().where(
            $"summary" === "25%").select(feature).first().mkString.toFloat
        var thirdQ = creditRiskdf.select(feature).summary().where(
            $"summary" === "75%").select(feature).first().mkString.toFloat
        //use formulas below to test threshold of outliers
        var testValHigh = thirdQ + (1.5 * (thirdQ - firstQ))
        var testValLow = firstQ - (1.5 * (thirdQ - firstQ))
        if(testValHigh == 0 & testValLow == 0){}
        //check to see if thresholds are exceeded in the column and count
        else{
            var outHigh = creditRiskdf.select(col(feature)).filter(
                col(feature) > lit(testValHigh)).count()
            var outLow = creditRiskdf.select(col(feature)).filter(
                col(feature) < lit(testValLow)).count()
            // collate a list of features with outliers > 5% of total data rows
            if (outHigh + outLow >= feature_dropThreshold ){       
                highOutlier_list = feature :: highOutlier_list
            }
            // collate a list of features with outliers 2%-5% of total data rows    
            else if (outHigh + outLow > 0.02*rows & outHigh + outLow < 0.05*rows){
                lowOutlier_list = feature :: lowOutlier_list
            }
        }
    }

    
}

highOutlier_list: List[String] = List()
lowOutlier_list: List[String] = List()
feature_dropThreshold: Double = 9193.75
row_dropLower: Double = 3677.5
outliers: (feature_set: Array[String], creditRiskdf: org.apache.spark.sql.DataFrame)Unit


In [18]:
// processing outliers list 1 of 7

val colNames = creditRiskFeatures1.map(col(_))
var creditRiskdf1_1 = creditRiskdf1.select(colNames:_*)
creditRiskdf1_1.cache()
outliers(creditRiskFeatures1, creditRiskdf1_1)
println(highOutlier_list.length)
println(lowOutlier_list.length)

creditRiskdf1_1.unpersist()

19
8


colNames: Array[org.apache.spark.sql.Column] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bur...

In [19]:
// processing outliers list 2 of 7

val colNames = creditRiskFeatures2.map(col(_))
var creditRiskdf1_2 = creditRiskdf1.select(colNames:_*)
creditRiskdf1_2.cache()
outliers(creditRiskFeatures2, creditRiskdf1_2)
println(highOutlier_list.length)
println(lowOutlier_list.length)
creditRiskdf1_2.unpersist()

36
16


colNames: Array[org.apache.spark.sql.Column] = Array(previous_loans_DAYS_DECISION_max, client_installments_AMT_INSTALMENT_min_mean, bureau_AMT_CREDIT_SUM_LIMIT_mean, bureau_DAYS_CREDIT_sum, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, previous_loans_PRODUCT_COMBINATION_Cash X-Sell: low_count_norm, previous_loans_HOUR_APPR_PROCESS_START_mean, bureau_AMT_CREDIT_SUM_DEBT_max, bureau_AMT_CREDIT_SUM_min, previous_loans_AMT_DOWN_PAYMENT_mean, NAME_FAMILY_STATUS_Married, previous_loans_CNT_PAYMENT_max, client_installments_AMT_INSTALMENT_mean_min, AMT_INCOME_TOTAL, previous_loans_DAYS_LAST_DUE_sum, client_cash_MONTHS_BALANCE_max_mean, NAME_EDUCATION_TYPE_Higher education, client_cash_MONTHS_BALANCE_sum_mean, bureau_DAYS_CREDIT_ENDDATE_mean, REGION_RATING_CLIENT, previous_loans_AMT_GOODS_PRICE_...

In [20]:
// processing outliers list 3 of 7

val colNames = creditRiskFeatures3.map(col(_))
var creditRiskdf1_3 = creditRiskdf1.select(colNames:_*)
creditRiskdf1_3.cache()
outliers(creditRiskFeatures3, creditRiskdf1_3)
println(highOutlier_list.length)
println(lowOutlier_list.length)
creditRiskdf1_3.unpersist()

51
20


colNames: Array[org.apache.spark.sql.Column] = Array(bureau_CREDIT_ACTIVE_Active_count_norm, bureau_DAYS_CREDIT_UPDATE_min, DEF_60_CNT_SOCIAL_CIRCLE, FLAG_DOCUMENT_3, bureau_DAYS_ENDDATE_FACT_mean, bureau_AMT_CREDIT_SUM_LIMIT_max, client_cash_SK_DPD_max_sum, previous_loans_AMT_GOODS_PRICE_mean, client_installments_AMT_PAYMENT_max_sum, client_installments_NUM_INSTALMENT_VERSION_sum_mean, client_installments_AMT_INSTALMENT_min_min, previous_loans_NAME_TYPE_SUITE_Unaccompanied_count_norm, client_cash_NAME_CONTRACT_STATUS_Active_count_norm_mean, client_installments_NUM_INSTALMENT_NUMBER_max_mean, client_cash_CNT_INSTALMENT_max_max, client_installments_DAYS_ENTRY_PAYMENT_max_min, client_installments_AMT_PAYMENT_max_mean, client_cash_counts_sum, previous_loans_NAME_CONTRACT_STATUS_Approved_co...

In [24]:
// persist storage of intermediate lists

import java.io._

val file1 = "highOutlier_list.txt"
val file2 = "lowOutlier_list.txt"
val writer1 = new BufferedWriter(new FileWriter(file1))
highOutlier_list.map(_+"\n").foreach(writer1.write)
writer1.close()
val writer2 = new BufferedWriter(new FileWriter(file2))
lowOutlier_list.map(_+"\n").foreach(writer2.write)
writer2.close()

import java.io._
file1: String = highOutlier_list.txt
file2: String = lowOutlier_list.txt
writer1: java.io.BufferedWriter = java.io.BufferedWriter@1db7651f
writer2: java.io.BufferedWriter = java.io.BufferedWriter@5d2426f6


In [14]:
// load intermediate lists
import scala.io.Source

var highOutlier_list = Source.fromFile("highOutlier_list.txt").getLines.toList
var lowOutlier_list = Source.fromFile("lowOutlier_list.txt").getLines.toList
var feature_dropThreshold = 0.05*rows
var row_dropLower = 0.02*rows

def outliers(feature_set: Array[String],creditRiskdf:org.apache.spark.sql.DataFrame){    
    for (feature <- feature_set) {
        //summarise column and query the DataFrame output
        var firstQ = creditRiskdf.select(feature).summary().where(
            $"summary" === "25%").select(feature).first().mkString.toFloat
        var thirdQ = creditRiskdf.select(feature).summary().where(
            $"summary" === "75%").select(feature).first().mkString.toFloat
        //use formulas below to test threshold of outliers
        var testValHigh = thirdQ + (1.5 * (thirdQ - firstQ))
        var testValLow = firstQ - (1.5 * (thirdQ - firstQ))
        if(testValHigh == 0 & testValLow == 0){}
        //check to see if thresholds are exceeded in the column and count
        else{
            var outHigh = creditRiskdf.select(col(feature)).filter(
                col(feature) > lit(testValHigh)).count()
            var outLow = creditRiskdf.select(col(feature)).filter(
                col(feature) < lit(testValLow)).count()
            // collate a list of features with outliers > 5% of total data rows
            if (outHigh + outLow >= feature_dropThreshold ){       
                highOutlier_list = feature :: highOutlier_list
            }
            // collate a list of features with outliers 2%-5% of total data rows    
            else if (outHigh + outLow > 0.02*rows & outHigh + outLow < 0.05*rows){
                lowOutlier_list = feature :: lowOutlier_list
            }
        }
    }

    
}

import scala.io.Source
highOutlier_list: List[String] = List(previous_loans_AMT_CREDIT_min, previous_loans_CNT_PAYMENT_sum, previous_loans_DAYS_LAST_DUE_1ST_VERSION_mean, client_installments_AMT_PAYMENT_mean_sum, previous_loans_AMT_ANNUITY_sum, client_installments_AMT_INSTALMENT_sum_max, client_cash_CNT_INSTALMENT_mean_min, client_cash_counts_sum, client_installments_AMT_PAYMENT_max_mean, client_cash_CNT_INSTALMENT_max_max, client_installments_NUM_INSTALMENT_NUMBER_max_mean, client_installments_AMT_INSTALMENT_min_min, client_installments_NUM_INSTALMENT_VERSION_sum_mean, client_installments_AMT_PAYMENT_max_sum, previous_loans_AMT_GOODS_PRICE_mean, client_cash_MONTHS_BALANCE_sum_max, client_installments_AMT_INSTALMENT_mean_max, bureau_DAYS_CREDIT_ENDDATE_sum, previous_loans_NAME_YIELD_GRO...

In [15]:
// processing outliers list 4 of 7

val colNames = creditRiskFeatures4.map(col(_))
var creditRiskdf1_4 = creditRiskdf1.select(colNames:_*)
creditRiskdf1_4.cache()
outliers(creditRiskFeatures4, creditRiskdf1_4)
println(highOutlier_list.length)
println(lowOutlier_list.length)
creditRiskdf1_4.unpersist()

71
25


colNames: Array[org.apache.spark.sql.Column] = Array(client_bureau_balance_MONTHS_BALANCE_mean_sum, previous_loans_PRODUCT_COMBINATION_Cash Street: high_count_norm, previous_loans_RATE_DOWN_PAYMENT_mean, bureau_AMT_CREDIT_SUM_DEBT_min, NAME_CONTRACT_TYPE_Cash loans, client_installments_AMT_INSTALMENT_min_sum, previous_loans_AMT_APPLICATION_max, client_installments_NUM_INSTALMENT_VERSION_sum_max, previous_loans_RATE_DOWN_PAYMENT_min, client_installments_AMT_INSTALMENT_mean_mean, client_installments_NUM_INSTALMENT_VERSION_sum_sum, previous_loans_NAME_TYPE_SUITE_Family_count_norm, previous_loans_DAYS_TERMINATION_mean, client_cash_CNT_INSTALMENT_FUTURE_max_sum, previous_loans_NAME_PAYMENT_TYPE_XNA_count_norm, bureau_CREDIT_TYPE_Credit card_count_norm, client_installments_DAYS_ENTRY_PAYMENT_...

In [16]:
// processing outliers list 5 of 7

val colNames = creditRiskFeatures5.map(col(_))
var creditRiskdf1_5 = creditRiskdf1.select(colNames:_*)
creditRiskdf1_5.cache()
outliers(creditRiskFeatures5, creditRiskdf1_5)
println(highOutlier_list.length)
println(lowOutlier_list.length)
creditRiskdf1_5.unpersist()

79
32


colNames: Array[org.apache.spark.sql.Column] = Array(client_bureau_balance_STATUS_0_count_sum, OCCUPATION_TYPE_Core staff, previous_loans_NAME_YIELD_GROUP_middle_count_norm, client_cash_NAME_CONTRACT_STATUS_Completed_count_norm_mean, bureau_AMT_CREDIT_SUM_OVERDUE_max, REG_CITY_NOT_LIVE_CITY, ORGANIZATION_TYPE_Business Entity Type 3, bureau_CREDIT_ACTIVE_Closed_count, ORGANIZATION_TYPE_Self-employed, previous_loans_NAME_PORTFOLIO_Cash_count_norm, previous_loans_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_count_norm, client_installments_NUM_INSTALMENT_VERSION_mean_min, client_cash_NAME_CONTRACT_STATUS_Completed_count_sum, previous_loans_CODE_REJECT_REASON_SCO_count_norm, previous_loans_NAME_GOODS_CATEGORY_Computers_count_norm, previous_loans_NAME_YIELD_GROUP_XNA_count_norm, previous_loans_PRODUC...

In [17]:
// processing outliers list 6 of 7

val colNames = creditRiskFeatures6.map(col(_))
var creditRiskdf1_6 = creditRiskdf1.select(colNames:_*)
creditRiskdf1_6.cache()
outliers(creditRiskFeatures6, creditRiskdf1_6)
println(highOutlier_list.length)
println(lowOutlier_list.length)
creditRiskdf1_6.unpersist()

86
41


colNames: Array[org.apache.spark.sql.Column] = Array(client_cash_CNT_INSTALMENT_min_max, previous_loans_WEEKDAY_APPR_PROCESS_START_THURSDAY_count_norm, ORGANIZATION_TYPE_Construction, client_cash_NAME_CONTRACT_STATUS_Active_count_max, client_cash_SK_DPD_DEF_sum_min, previous_loans_NAME_CONTRACT_TYPE_Cash loans_count_norm, previous_loans_DAYS_FIRST_DRAWING_mean, previous_loans_PRODUCT_COMBINATION_Cash X-Sell: middle_count_norm, previous_loans_NAME_CASH_LOAN_PURPOSE_Other_count_norm, client_bureau_balance_STATUS_0_count_norm_sum, client_installments_NUM_INSTALMENT_VERSION_sum_min, previous_loans_CHANNEL_TYPE_Country-wide_count_norm, client_cash_NAME_CONTRACT_STATUS_Signed_count_sum, AMT_REQ_CREDIT_BUREAU_YEAR, bureau_CREDIT_TYPE_Microloan_count, NAME_INCOME_TYPE_State servant, previous_lo...

In [18]:
// processing outliers list 7 of 7

val colNames = creditRiskFeatures7.map(col(_))
var creditRiskdf1_7 = creditRiskdf1.select(colNames:_*)
creditRiskdf1_7.cache()
outliers(creditRiskFeatures7, creditRiskdf1_7)
println(highOutlier_list.length)
println(lowOutlier_list.length)
creditRiskdf1_7.unpersist()

94
47


colNames: Array[org.apache.spark.sql.Column] = Array(previous_loans_PRODUCT_COMBINATION_Cash Street: middle_count_norm, previous_loans_PRODUCT_COMBINATION_POS other with interest_count_norm, client_cash_CNT_INSTALMENT_min_min, previous_loans_CODE_REJECT_REASON_VERIF_count_norm, previous_loans_CODE_REJECT_REASON_LIMIT_count_norm, client_bureau_balance_STATUS_X_count_norm_sum, previous_loans_PRODUCT_COMBINATION_Cash_count_norm, previous_loans_NAME_GOODS_CATEGORY_Consumer Electronics_count_norm, previous_loans_CHANNEL_TYPE_Regional / Local_count_norm, previous_loans_WEEKDAY_APPR_PROCESS_START_SUNDAY_count, FLAG_DOCUMENT_13, previous_loans_CODE_REJECT_REASON_CLIENT_count_norm, client_bureau_balance_STATUS_X_count_sum, previous_loans_NAME_YIELD_GROUP_middle_count, previous_loans_NAME_CONTRAC...

In [19]:
// persist storage of intermediate lists

import java.io._

val file1 = "highOutlier_FinalList.txt"
val file2 = "lowOutlier_FinalList.txt"
val writer1 = new BufferedWriter(new FileWriter(file1))
highOutlier_list.map(_+"\n").foreach(writer1.write)
writer1.close()
val writer2 = new BufferedWriter(new FileWriter(file2))
lowOutlier_list.map(_+"\n").foreach(writer2.write)
writer2.close()

import java.io._
file1: String = highOutlier_FinalList.txt
file2: String = lowOutlier_FinalList.txt
writer1: java.io.BufferedWriter = java.io.BufferedWriter@bb849c
writer2: java.io.BufferedWriter = java.io.BufferedWriter@7f253bd8


### Continue from cells below if directly loading highOutlier and lowOutlier lists from text files.

In [14]:
import scala.io.Source

var highOutlier_list = Source.fromFile("highOutlier_FinalList.txt").getLines.toList
var lowOutlier_list = Source.fromFile("lowOutlier_FinalList.txt").getLines.toList

import scala.io.Source
highOutlier_list: List[String] = List(client_bureau_balance_MONTHS_BALANCE_max_sum, previous_loans_NAME_CONTRACT_STATUS_Refused_count, NAME_HOUSING_TYPE_House / apartment, previous_loans_NAME_PRODUCT_TYPE_XNA_count, previous_loans_NAME_CONTRACT_TYPE_Revolving loans_count_norm, client_bureau_balance_STATUS_X_count_sum, previous_loans_NAME_GOODS_CATEGORY_Consumer Electronics_count_norm, client_bureau_balance_STATUS_X_count_norm_sum, previous_loans_NAME_GOODS_CATEGORY_Mobile_count_norm, previous_loans_PRODUCT_COMBINATION_POS household with interest_count_norm, previous_loans_WEEKDAY_APPR_PROCESS_START_THURSDAY_count, client_bureau_balance_STATUS_0_count_norm_sum, previous_loans_PRODUCT_COMBINATION_Cash X-Sell: middle_count_norm, previous_loans_DAYS_FIRST_DRAWING_mean...

In [15]:
println(highOutlier_list.length)
println(lowOutlier_list.length)
creditRiskdf.unpersist()

94
47


res5: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 280 more fields]


Loaded lists has identical rows to outliers variables processed earlier.

In [16]:
creditRiskdf1.cache()

res6: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 280 more fields]


In [17]:
// drop features in highOutlier_list
creditRiskdf1 = creditRiskdf1.drop(highOutlier_list:_*)

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [18]:
println(creditRiskdf1.count())
println(creditRiskdf1.columns.size)

183875
188


### **Output**
creditRiskdf1 - cleansed dataset with features with outliers >5% removed. 

In [33]:
var creditRiskdf2 = creditRiskdf1

creditRiskdf2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [20]:
creditRiskdf2.cache()

res8: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [34]:
//code block takes features in lowOutlier_list and filters main dataframe against them
//code will generate a subset of main dataset without outliers.

//input feature list conatining features with outliers persisting

for (feature <- lowOutlier_list){
    //summarise column and query the DataFrame output
    var firstQ = creditRiskdf2.select(feature).summary().where(
        $"summary" === "25%").select(feature).first().mkString.toFloat
    var thirdQ = creditRiskdf2.select(feature).summary().where(
        $"summary" === "75%").select(feature).first().mkString.toFloat
    //use formulas below to test threshold of outliers
    var testValHigh = thirdQ + (1.5 * (thirdQ - firstQ))
    var testValLow = firstQ - (1.5 * (thirdQ - firstQ))
    if (testValHigh == 0 & testValLow == 0){}
    else {
        creditRiskdf2 = creditRiskdf2.filter(col(feature) <= lit(testValHigh)).filter(
            col(feature) > lit(testValLow))        
    }  
}

In [35]:
println(creditRiskdf2.count())
println(creditRiskdf2.columns.size)

35583
188


### **Output**
creditRiskdf2 - cleansed dataset with features with outliers >5% removed and data rows for features with outliers between 2-5% removed.

In [23]:
creditRiskdf1.write.option("header", "true").csv("creditRiskdf1.csv")

In [36]:
creditRiskdf2.write.option("header", "true").csv("creditRiskdf2.csv")

In [25]:
creditRiskdf1.unpersist()
creditRiskdf2.unpersist()

res13: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


## 3.4 Machine Learning

### Load the csv files to skip the outlier processing time

In [14]:

// Load creditRiskdf1 datasets: which exclude features with outliers >5%

val creditRiskdf1_1 = spark.read.option("header","true").csv(
    "creditRiskdf1_1.csv")
val creditRiskdf1_2 = spark.read.option("header","true").csv(
    "creditRiskdf1_2.csv")
val creditRiskdf1_3 = spark.read.option("header","true").csv(
    "creditRiskdf1_3.csv")


creditRiskdf1_1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf1_2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf1_3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]


In [15]:

// combine the 3 dataframes to 1

val creditRiskdf1and2 = creditRiskdf1_1.union(creditRiskdf1_2)
var creditRiskdf1 = creditRiskdf1and2.union(creditRiskdf1_3)


creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]


In [16]:

// Load creditRiskdf2 datasets: which exclude features with outliers >5% 
// and datarows of features with outliers between 2-5%

val creditRiskdf2_1 = spark.read.option("header","true").csv(
    "creditRiskdf2_1.csv")
val creditRiskdf2_2 = spark.read.option("header","true").csv(
    "creditRiskdf2_2.csv")
val creditRiskdf2_3 = spark.read.option("header","true").csv(
    "creditRiskdf2_3.csv")


creditRiskdf2_1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf2_2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf2_3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]


In [17]:

// combine the 3 dataframes to 1

val creditRiskdf1and2 = creditRiskdf2_1.union(creditRiskdf2_2)
var creditRiskdf2= creditRiskdf1and2.union(creditRiskdf2_3)


creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]
creditRiskdf2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]


In [18]:

creditRiskdf1.cache()
creditRiskdf2.cache()


res5: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 186 more fields]


In [19]:

println("creditRiskdf1 structure:")
println(creditRiskdf1.count())
println(creditRiskdf1.columns.size)
println("creditRiskdf2 structure:")
println(creditRiskdf2.count())
println(creditRiskdf2.columns.size)


creditRiskdf1 structure:
183875
188
creditRiskdf2 structure:
35583
188


In [20]:

// change datatype of data from String to Float
var creditRiskFeatures1 = creditRiskdf1.columns
var creditRiskFeatures2 = creditRiskdf2.columns

for (colName<-creditRiskFeatures1){
     |   creditRiskdf1 = creditRiskdf1.withColumn(
         colName,col(colName).cast("Float"))
     | }

for (colName<-creditRiskFeatures2){
     |   creditRiskdf2 = creditRiskdf2.withColumn(
         colName,col(colName).cast("Float"))
     | }


creditRiskFeatures1: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_ID_PUBLISH, previous_loans_CNT_PAYMENT_mean, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, DAYS_REGISTRATION, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_mean, previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm, client_installments_NUM_INSTALMENT_VERSION_max_mean, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_RATE_DOWN_PAYMENT_sum, previous_loans_DAYS_FIRST_DUE_min, client_cash_SK_DPD_DEF_sum_max, bureau_AMT_CREDIT_SUM_LIMIT_mean, bureau_DAYS_CREDIT_sum, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, previous_loans_PRODUCT_COMBINATION_Cash X-Sell: low_count_norm,...

In [21]:
creditRiskdf1.unpersist()
creditRiskdf2.unpersist()

res8: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


### **Outputs**
- creditRiskdf1: Cleansed dataset - features with outliers > 5% data rows removed.
- creditRiskdf2 - cleansed dataset with features with outliers >5% removed and data rows for features with outliers between 2-5% removed.

### 3.4.1 Prepare training, validation and test sets

In [22]:
// random split of cleansed dataset to training set (70%), testing set (20%)
// and validation set (10%)

var Array(train_df1,test_df1,validation_df1) = creditRiskdf1.randomSplit(
    Array(0.7,0.2,0.1))
var Array(train_df2,test_df2,validation_df2) = creditRiskdf2.randomSplit(
    Array(0.7,0.2,0.1))

train_df1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
test_df1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
validation_df1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
train_df2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
test_df2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
validation_df2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


**Outputs**
- train_df1: to be used for fitting model1.
- validation_df1: to be used for evaluating model1.
- test_df1: to be kept hidden throughout fitting process and used to test the final model1.
- train_df2: to be used for fitting model2.
- validation_df2: to be used for evaluating model2.
- test_df2: to be kept hidden throughout fitting process and used to test the final model2.

### 3.4.2 Model fitting and evaluation

In [23]:
train_df1.cache()
test_df1.cache()
validation_df1.cache()

res9: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [24]:
// collate arrays of column names with and without the label "TARGET"

val train_df1Features = train_df1.columns
var features = train_df1Features.filter(!_.contains("TARGET"))
features = features.filter(!_.contains("SK_ID_CURR"))

train_df1Features: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_ID_PUBLISH, previous_loans_CNT_PAYMENT_mean, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, DAYS_REGISTRATION, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_mean, previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm, client_installments_NUM_INSTALMENT_VERSION_max_mean, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_RATE_DOWN_PAYMENT_sum, previous_loans_DAYS_FIRST_DUE_min, client_cash_SK_DPD_DEF_sum_max, bureau_AMT_CREDIT_SUM_LIMIT_mean, bureau_DAYS_CREDIT_sum, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, previous_loans_PRODUCT_COMBINATION_Cash X-Sell: low_count_norm, p...

In [25]:
// ensure that "features" does not contain SK_ID_CURR, TARGET or label
features.filter(x => x == "SK_ID_CURR" | x == "TARGET" | x == "label")

res10: Array[String] = Array()


### 3.4.2.1 Binomial Logistic Regression

In [26]:
// place all features except for "TARGET" under column name "features"

val lrAssembler = new VectorAssembler().setInputCols(features).setOutputCol(
    "features")

lrAssembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_46f2062de2b1


In [27]:
// identify label for logistic regression

val lrLabelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol(
    "label")

lrLabelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_a75f174614e8


In [28]:
// fit logistic model

val modelLr = new LogisticRegression()

modelLr: org.apache.spark.ml.classification.LogisticRegression = logreg_917f071b016d


#### Prepare pipeline for future validation, optimisation and testing.

In [29]:
val lrPipeline = new Pipeline().setStages(Array(
    lrLabelIndexer, lrAssembler, modelLr))

lrPipeline: org.apache.spark.ml.Pipeline = pipeline_092f8c8273b1


In [30]:
val lrModel = lrPipeline.fit(train_df1)

2019-06-09 09:41:10,913 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2019-06-09 09:41:10,921 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


lrModel: org.apache.spark.ml.PipelineModel = pipeline_092f8c8273b1


#### Evaluate Logistic Regression Model

Determine the metric to evaluate the model, e.g. ROC AUC.

#### Predict using validation data

In [31]:
val predictions1 = lrModel.transform(validation_df1)
predictions1.select ("SK_ID_CURR","features", "label","probability", "prediction").show()

+----------+--------------------+-----+--------------------+----------+
|SK_ID_CURR|            features|label|         probability|prediction|
+----------+--------------------+-----+--------------------+----------+
|  147356.0|(186,[0,1,2,3,4,5...|  0.0|[0.89727683521013...|       0.0|
|  125091.0|(186,[0,1,2,3,4,5...|  1.0|[0.59622894388730...|       0.0|
|  125159.0|(186,[0,1,2,3,4,5...|  0.0|[0.54393536048157...|       0.0|
|  116133.0|(186,[0,1,2,3,4,5...|  1.0|[0.86694266148273...|       0.0|
|  139376.0|(186,[0,1,2,3,4,5...|  0.0|[0.91901451770221...|       0.0|
|  162796.0|(186,[0,1,2,3,4,5...|  0.0|[0.67739912162574...|       0.0|
|  169646.0|(186,[0,1,2,3,4,5...|  0.0|[0.62198363722784...|       0.0|
|  181824.0|(186,[0,1,2,3,4,5...|  1.0|[0.71096236422142...|       0.0|
|  153344.0|(186,[0,1,2,3,4,5...|  0.0|[0.57556155290442...|       0.0|
|  206971.0|(186,[0,1,2,3,4,5...|  0.0|[0.92910830381305...|       0.0|
|  210708.0|(186,[0,1,2,3,4,5...|  0.0|[0.92689831260893...|    

predictions1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 191 more fields]


In [32]:
// println(s"Coefficients: ${model.coefficients} Intercept: ${model.intercept}")

#### Accuracy measurement 1

In [33]:
import org.apache.spark.ml.classification.NaiveBayes
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.sql.SparkSession

import org.apache.spark.ml.classification.NaiveBayes
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql.SparkSession


In [34]:
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions1)
println("Test set accuracy = " + accuracy)

Test set accuracy = 0.9258536585365854


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_371445436a86
accuracy: Double = 0.9258536585365854


We can see that the accuracy of our prediction is over 92.58%. Seems pretty good(?)

In [35]:
val t1: Float = validation_df1.filter($"TARGET" ===  1).count()
val t0: Float = validation_df1.filter($"TARGET" ===  0).count()
val t1_percent = t1/validation_df1.count()
val t0_percent = t0/validation_df1.count()

t1: Float = 1365.0
t0: Float = 17085.0
t1_percent: Float = 0.07398374
t0_percent: Float = 0.9260163


The underlying data is split 7.40% and 92.60% for TARGET 1 and 0 respectively. Although the accuracy of the model is 92.58%, it is predicting marginally worse than the statistic means of the dataset.

#### Accuracy measurement 2

In [36]:
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics


In [37]:
val lp = predictions1.select( "label", "prediction")
val counttotal = predictions1.count()
val correct = lp.filter($"label" === $"prediction").count() //correct prediction
val wrong = lp.filter(not($"label" === $"prediction")).count() //wrong prediction
val truep = lp.filter($"prediction" === 0.0).filter($"label" === $"prediction").count() //Correct prediction of label=0
val falseN = lp.filter($"prediction" === 0.0).filter(not($"label" === $"prediction")).count() //Wrong prediction when label=1
val falseP = lp.filter($"prediction" === 1.0).filter(not($"label" === $"prediction")).count() //Wrong prediction when lable =0
val ratioWrong=wrong.toDouble/counttotal.toDouble //Wrong prediction ratio
val ratioCorrect=correct.toDouble/counttotal.toDouble //Correct predicition ratio

lp: org.apache.spark.sql.DataFrame = [label: double, prediction: double]
counttotal: Long = 18450
correct: Long = 17082
wrong: Long = 1368
truep: Long = 17055
falseN: Long = 1338
falseP: Long = 30
ratioWrong: Double = 0.07414634146341463
ratioCorrect: Double = 0.9258536585365854


#### Accuracy measurement 3

In [38]:
val predictionLabelsRDD = predictions1.select("prediction", "label").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics = new BinaryClassificationMetrics(predictionLabelsRDD)

predictionLabelsRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[453] at map at <console>:48
binMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@67ec21c2


In [39]:
println("Area under the receiver operating characteristic (ROC) curve : " + binMetrics.areaUnderROC)

Area under the receiver operating characteristic (ROC) curve : 0.5090121467645612


The AUC ROC of 0.51 is indicating the logistic regression model has not provided a strong class separation.

### 3.4.2.2 Random Forest

In [40]:
/*
// Cross check training data set is being used
println("Total data rows in combined dataframe: "+ train_df1.count())
var RF_df = train_df1
val train_df1Features = train_df1.columns
for (colName<-creditRiskFeatures){
     |   RF_df=RF_df.withColumn(colName,col(colName).cast("Double"))
     | }
//RF_df.printSchema()
*/

In [41]:
var Array(train_df1,test_df1,validation_df1)=creditRiskdf1.randomSplit(
    Array(0.7,0.2,0.1))

train_df1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
test_df1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
validation_df1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [42]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}


In [43]:
val c1 = train_df1

c1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [44]:
// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.
val rfLabelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol(
    "indexedLabel").fit(c1)

rfLabelIndexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_484f47968833


In [45]:
val rfAssembler = new VectorAssembler().setInputCols(features).setOutputCol(
    "features")
val df1 = rfAssembler.transform(c1)

rfAssembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_df99d5945cca
df1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 187 more fields]


In [46]:
// Automatically identify categorical features, and index them.
// Set maxCategories so features with > 4 distinct values are treated as continuous.
val rfFeatureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .setMaxCategories(4).fit(df1)

rfFeatureIndexer: org.apache.spark.ml.feature.VectorIndexerModel = vecIdx_6461f536c3df


In [47]:
// Train a RandomForest model.
val modelRf = new RandomForestClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("indexedFeatures")  
  .setNumTrees(20)

modelRf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_5c511328a8c1


In [48]:
// Convert indexed labels back to original labels.
val rfLabelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(rfLabelIndexer.labels)

rfLabelConverter: org.apache.spark.ml.feature.IndexToString = idxToStr_41a5791c024a


In [49]:
// Chain indexers and forest in a Pipeline.
val rfPipeline = new Pipeline()
  .setStages(Array(rfLabelIndexer, rfFeatureIndexer, modelRf, rfLabelConverter))

rfPipeline: org.apache.spark.ml.Pipeline = pipeline_f17db21b15b0


In [50]:
// Train model. This also runs the indexers.
val rfModel = rfPipeline.fit(df1)

rfModel: org.apache.spark.ml.PipelineModel = pipeline_f17db21b15b0


#### Evaluate Random Forest Model

In [51]:
val df2 = rfAssembler.transform(validation_df1)

// Make predictions.
val predictions2 = rfModel.transform(df2)

df2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 187 more fields]
predictions2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 193 more fields]


In [52]:
// Select example rows to display.

predictions2.select("features", "indexedLabel", "probability","predictedLabel").show(20)

+--------------------+------------+--------------------+--------------+
|            features|indexedLabel|         probability|predictedLabel|
+--------------------+------------+--------------------+--------------+
|(186,[0,1,2,3,4,5...|         0.0|[0.91563597649114...|           0.0|
|(186,[0,1,2,3,4,5...|         0.0|[0.91590035869383...|           0.0|
|(186,[0,1,2,3,4,5...|         0.0|[0.92766246019460...|           0.0|
|(186,[0,1,2,3,4,5...|         1.0|[0.85817118937111...|           0.0|
|(186,[0,1,2,3,4,5...|         0.0|[0.91994216494559...|           0.0|
|(186,[0,1,2,3,4,5...|         1.0|[0.91543889284909...|           0.0|
|(186,[0,1,2,3,4,5...|         0.0|[0.92249828864000...|           0.0|
|(186,[0,1,2,3,4,5...|         1.0|[0.90517976178519...|           0.0|
|(186,[0,1,2,3,4,5...|         0.0|[0.92491810062623...|           0.0|
|(186,[0,1,2,3,4,5...|         1.0|[0.88952982283765...|           0.0|
|(186,[0,1,2,3,4,5...|         0.0|[0.92429906460496...|        

#### Accuracy measurement 1

In [53]:
// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions2)
println(s"Test Error = ${(1.0 - accuracy)}")

Test Error = 0.07264887287811805


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_8a8c5f50906a
accuracy: Double = 0.927351127121882


We can see that the accuracy of our prediction is 92.74%. However, as noted with the logistic regression, this accuracy appears to be marginally better than the dataset means.

#### Accuracy measurement 2

In [54]:
val predictionLabelsRDD = predictions2.select("prediction", "indexedLabel").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics = new BinaryClassificationMetrics(predictionLabelsRDD)

predictionLabelsRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[589] at map at <console>:54
binMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@63a9476f


In [55]:
println("Area under the receiver operating characteristic (ROC) curve : " + binMetrics.areaUnderROC)

Area under the receiver operating characteristic (ROC) curve : 0.5


At an AUC ROC of 0.5, the Random forest classifier has not yielded separation of the classes.

In [56]:
train_df1.unpersist()
test_df1.unpersist()
validation_df1.unpersist()

res19: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


### 3.4.2.3 Multilayer Perceptron Model

In [57]:
import org.apache.spark.ml.classification.MultilayerPerceptronClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.ml.classification.MultilayerPerceptronClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator


In [58]:
var dfTrain = train_df1.withColumnRenamed("TARGET", "label")
dfTrain = dfTrain.drop("SK_ID_CURR")

var dfTest = test_df1.withColumnRenamed("TARGET", "label")
dfTest = dfTest.drop("SK_ID_CURR")

dfTrain: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]
dfTrain: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]
dfTest: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]
dfTest: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]


In [59]:
dfTrain.cache()
dfTest.cache()

res20: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]


In [60]:
println("Training set count: "+dfTrain.count() + "\n" +
        "Test set count: "+dfTest.count())
println("Training column count: "+dfTrain.columns.size + "\n" +
        "Test column count: "+dfTest.columns.size)

Training set count: 128324
Test set count: 37230
Training column count: 187
Test column count: 187


In [61]:
// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.
var mlpcLabelIndexer = new StringIndexer().setInputCol("label").setOutputCol(
    "indexedLabel").fit(dfTrain)

var mlpcAssembler = new VectorAssembler()
    .setInputCols(features).setOutputCol("features")

//var mlpcTrain = Assembler.transform(dfTrain)

mlpcLabelIndexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_56d4a92d6718
mlpcAssembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_08b91696fadb


In [67]:
// Automatically identify categorical features, and index them.
//var mlpcFeatureIndexer = new VectorIndexer()
//  .setInputCol("features")
//  .setOutputCol("indexedFeatures")

// specify layers for the neural network:
// input layer of size 187 (features), two intermediate of size 45 and 10
// and output of size 2 (classes)
//https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/

var layers = Array[Int](186, 50, 25, 2)

// create the trainer and set its parameters
var mlpcModel = new MultilayerPerceptronClassifier()
  .setLayers(layers)
  .setBlockSize(128)
  .setTol(0.0005)
  .setSeed(123)
  .setMaxIter(700)

layers: Array[Int] = Array(186, 50, 25, 2)
mlpcModel: org.apache.spark.ml.classification.MultilayerPerceptronClassifier = mlpc_3ec4a2453930


In [68]:
// Convert indexed labels back to original labels.
val mlpcLabelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(mlpcLabelIndexer.labels)

// Chain indexers and forest in a Pipeline.
var mlpcPipeline = new Pipeline().setStages(
    Array(mlpcLabelIndexer, mlpcAssembler, mlpcModel, mlpcLabelConverter))


mlpcLabelConverter: org.apache.spark.ml.feature.IndexToString = idxToStr_79d8010d7ec5
mlpcPipeline: org.apache.spark.ml.Pipeline = pipeline_c31d42af4b3a


In [69]:
// train the model
var mlpcModelTrained = mlpcPipeline.fit(dfTrain)

mlpcModelTrained: org.apache.spark.ml.PipelineModel = pipeline_c31d42af4b3a


In [70]:
var result = mlpcModelTrained.transform(dfTest)

result: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 191 more fields]


In [71]:
result.printSchema

root
 |-- EXT_SOURCE_2: float (nullable = true)
 |-- EXT_SOURCE_3: float (nullable = true)
 |-- DAYS_BIRTH: float (nullable = true)
 |-- AMT_CREDIT: float (nullable = true)
 |-- AMT_ANNUITY: float (nullable = true)
 |-- DAYS_ID_PUBLISH: float (nullable = true)
 |-- previous_loans_CNT_PAYMENT_mean: float (nullable = true)
 |-- previous_loans_SELLERPLACE_AREA_max: float (nullable = true)
 |-- DAYS_LAST_PHONE_CHANGE: float (nullable = true)
 |-- DAYS_REGISTRATION: float (nullable = true)
 |-- CODE_GENDER_F: float (nullable = true)
 |-- REGION_POPULATION_RELATIVE: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_max_mean: float (nullable = true)
 |-- bureau_DAYS_CREDIT_mean: float (nullable = true)
 |-- previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm: float (nullable = true)
 |-- client_installments_NUM_INSTALMENT_VERSION_max_mean: float (nullable = true)
 |-- bureau_DAYS_CREDIT_min: float (nullable = true)
 |-- previous_loans_NAME_YIELD_GROUP_high_count_norm: float (nullable

In [72]:
result.select("features", "label", "probability","prediction").show(20)

+--------------------+-----+--------------------+----------+
|            features|label|         probability|prediction|
+--------------------+-----+--------------------+----------+
|(186,[0,1,2,3,4,5...|  0.0|[0.91647943660435...|       0.0|
|(186,[0,1,2,3,4,5...|  0.0|[0.92927620747784...|       0.0|
|(186,[0,1,2,3,4,5...|  1.0|[0.92810015207010...|       0.0|
|(186,[0,1,2,3,4,5...|  0.0|[0.94362794007198...|       0.0|
|(186,[0,1,2,3,4,5...|  0.0|[0.92892181267675...|       0.0|
|(186,[0,1,2,3,4,5...|  0.0|[0.91408102866485...|       0.0|
|(186,[0,1,2,3,4,5...|  1.0|[0.94448131627065...|       0.0|
|(186,[0,1,2,3,4,5...|  0.0|[0.93883166118189...|       0.0|
|(186,[0,1,2,3,4,5...|  0.0|[0.92601032327863...|       0.0|
|(186,[0,1,2,3,4,5...|  0.0|[0.92683238317496...|       0.0|
|(186,[0,1,2,3,4,5...|  1.0|[0.93545668765673...|       0.0|
|(186,[0,1,2,3,4,5...|  1.0|[0.92410409078157...|       0.0|
|(186,[0,1,2,3,4,5...|  0.0|[0.92615580562018...|       0.0|
|(186,[0,1,2,3,4,5...|  

In [73]:
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

val accuracy = evaluator.evaluate(result)

println("Test set accuracy = " + accuracy)

Test set accuracy = 0.9284448025785657


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_1b0b13971df5
accuracy: Double = 0.9284448025785657


In [74]:
val predictionLabelsRDD = result.select("prediction", "indexedLabel").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics = new BinaryClassificationMetrics(predictionLabelsRDD)

predictionLabelsRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[768] at map at <console>:56
binMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@1ee42fd4


In [75]:
println("Area under the receiver operating characteristic (ROC) curve : " + binMetrics.areaUnderROC)

Area under the receiver operating characteristic (ROC) curve : 0.5


In [76]:
dfTrain.unpersist()
dfTest.unpersist()

res28: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]


### 3.4.2.4 Model fitting conclusions

**Binomial Logistic Regression**
- Accuracy: 92.58%
- AUC ROC: 0.51

**Random Forest**
- Accuracy: 92.74%
- AUC ROC: 0.50

**Multilayer Perceptron Classifier**
- Accuracy: 92.84%
- AUC ROC: 0.50

Although all 3 models generated high accuracy scores, they compared close to statistic means of the underlying data which is split 7.40% and 92.60% for TARGET 1 and 0 respectively. 

Using another measure AUC ROC, it indicates that the models were not able to generate further class separation of the potential defaulting and non-defaulting customers.  This is due to a high skew of dataset towards non-defaulting customers.


### 3.4.3 Model Optimisation 1 -  outlier curtailment

In this model optimisation, dataset creditRiskdf2 will be used. As a refresher, this dataset has outlier data rows for features with 2-5% outliers (as a % of total data rows) removed.

In [77]:
train_df2.cache()
test_df2.cache()
validation_df2.cache()

res29: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


### 3.4.3.1 Binomial Logistic Regression

In [78]:
val lrModel = lrPipeline.fit(train_df2)

lrModel: org.apache.spark.ml.PipelineModel = pipeline_092f8c8273b1


In [79]:
val predictions3 = lrModel.transform(validation_df2)
predictions1.select ("SK_ID_CURR","features", "label","probability", "prediction").show()

+----------+--------------------+-----+--------------------+----------+
|SK_ID_CURR|            features|label|         probability|prediction|
+----------+--------------------+-----+--------------------+----------+
|  147356.0|(186,[0,1,2,3,4,5...|  0.0|[0.89727683521013...|       0.0|
|  125091.0|(186,[0,1,2,3,4,5...|  1.0|[0.59622894388730...|       0.0|
|  125159.0|(186,[0,1,2,3,4,5...|  0.0|[0.54393536048157...|       0.0|
|  116133.0|(186,[0,1,2,3,4,5...|  1.0|[0.86694266148273...|       0.0|
|  139376.0|(186,[0,1,2,3,4,5...|  0.0|[0.91901451770221...|       0.0|
|  162796.0|(186,[0,1,2,3,4,5...|  0.0|[0.67739912162574...|       0.0|
|  169646.0|(186,[0,1,2,3,4,5...|  0.0|[0.62198363722784...|       0.0|
|  181824.0|(186,[0,1,2,3,4,5...|  1.0|[0.71096236422142...|       0.0|
|  153344.0|(186,[0,1,2,3,4,5...|  0.0|[0.57556155290442...|       0.0|
|  206971.0|(186,[0,1,2,3,4,5...|  0.0|[0.92910830381305...|       0.0|
|  210708.0|(186,[0,1,2,3,4,5...|  0.0|[0.92689831260893...|    

predictions3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 191 more fields]


In [80]:
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions3)
println("Test set accuracy = " + accuracy)

Test set accuracy = 0.923054587688734


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_3997623069d8
accuracy: Double = 0.923054587688734


In [81]:
val t1: Float = validation_df1.filter($"TARGET" ===  1).count()
val t0: Float = validation_df1.filter($"TARGET" ===  0).count()
val t1_percent = t1/validation_df1.count()
val t0_percent = t0/validation_df1.count()

t1: Float = 1331.0
t0: Float = 16990.0
t1_percent: Float = 0.072648875
t0_percent: Float = 0.9273511


In [82]:
val lp = predictions3.select( "label", "prediction")
val counttotal = predictions1.count()
val correct = lp.filter($"label" === $"prediction").count() //correct prediction
val wrong = lp.filter(not($"label" === $"prediction")).count() //wrong prediction
val truep = lp.filter($"prediction" === 0.0).filter($"label" === $"prediction").count() //Correct prediction of label=0
val falseN = lp.filter($"prediction" === 0.0).filter(not($"label" === $"prediction")).count() //Wrong prediction when label=1
val falseP = lp.filter($"prediction" === 1.0).filter(not($"label" === $"prediction")).count() //Wrong prediction when lable =0
val ratioWrong=wrong.toDouble/counttotal.toDouble //Wrong prediction ratio
val ratioCorrect=correct.toDouble/counttotal.toDouble //Correct predicition ratio

lp: org.apache.spark.sql.DataFrame = [label: double, prediction: double]
counttotal: Long = 18450
correct: Long = 3179
wrong: Long = 265
truep: Long = 3177
falseN: Long = 261
falseP: Long = 4
ratioWrong: Double = 0.014363143631436315
ratioCorrect: Double = 0.17230352303523036


In [83]:
val predictionLabelsRDD = predictions3.select("prediction", "label").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics = new BinaryClassificationMetrics(predictionLabelsRDD)

predictionLabelsRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1096] at map at <console>:56
binMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@52b4af98


In [84]:
println("Area under the receiver operating characteristic (ROC) curve : " + binMetrics.areaUnderROC)

Area under the receiver operating characteristic (ROC) curve : 0.5031735482660235


AUC ROC for logistic regression model decreased from 0.51 to 0.50 using creditRiskdf2.

### 3.4.3.2 Random Forest

In [85]:
val c2 = train_df2

// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.
val rfLabelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol(
    "indexedLabel").fit(c2)

val df3 = rfAssembler.transform(c2)

// Train model. This also runs the indexers.
val rfModel = rfPipeline.fit(df3)

c2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
rfLabelIndexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_a5ea39c94308
df3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 187 more fields]
rfModel: org.apache.spark.ml.PipelineModel = pipeline_f17db21b15b0


In [86]:
val df4 = rfAssembler.transform(validation_df2)

// Make predictions.
val predictions4 = rfModel.transform(df4)

df4: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 187 more fields]
predictions4: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 193 more fields]


In [87]:
// Select example rows to display.

predictions4.select("features", "indexedLabel", "probability","predictedLabel").show(20)

+--------------------+------------+--------------------+--------------+
|            features|indexedLabel|         probability|predictedLabel|
+--------------------+------------+--------------------+--------------+
|(186,[0,1,2,3,4,5...|         0.0|[0.90582830218447...|           0.0|
|(186,[0,1,2,3,4,5...|         1.0|[0.90093370608547...|           0.0|
|(186,[0,1,2,3,4,5...|         1.0|[0.91498151754966...|           0.0|
|(186,[0,1,2,3,4,5...|         1.0|[0.93468911124092...|           0.0|
|(186,[0,1,2,3,4,5...|         0.0|[0.93519520871848...|           0.0|
|(186,[0,1,2,3,4,5...|         0.0|[0.91448862058862...|           0.0|
|(186,[0,1,2,3,4,5...|         0.0|[0.89567564996996...|           0.0|
|(186,[0,1,2,3,4,5...|         1.0|[0.90133440043016...|           0.0|
|(186,[0,1,2,3,4,5...|         0.0|[0.90184265770812...|           0.0|
|(186,[0,1,2,3,4,5...|         0.0|[0.87752822973581...|           0.0|
|(186,[0,1,2,3,4,5...|         0.0|[0.91688584539600...|        

In [88]:
// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions4)
println(s"Test Error = ${(1.0 - accuracy)}")

Test Error = 0.07636469221835074


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_ec570692784f
accuracy: Double = 0.9236353077816493


In [89]:
val predictionLabelsRDD = predictions4.select("prediction", "indexedLabel").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics = new BinaryClassificationMetrics(predictionLabelsRDD)

predictionLabelsRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1188] at map at <console>:56
binMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@53572ad7


In [90]:
println("Area under the receiver operating characteristic (ROC) curve : " + binMetrics.areaUnderROC)

Area under the receiver operating characteristic (ROC) curve : 0.5


No improvement. AUC ROC remained at 0.5.

In [91]:
/*
val rfModel = model.stages(2).asInstanceOf[RandomForestClassificationModel]
println(s"Learned classification forest model:\n ${rfModel.toDebugString}")

*/

### 3.4.3.3 Model Optimisation 1 conclusions

**Binomial Logistic Regression**
- Accuracy: 92.31%
- AUC ROC: 0.50

**Random Forest**
- Accuracy: 92.36%
- AUC ROC: 0.50

**Multilayer Perceptron Classifier**
- did not run as there was no evidence of optimisation happening with the logistic regression and random forest models

Again, the models generated high accuracy scores being trained with smaller dataset. However, they compared close to statistic means of the underlying data which is split 7.27% and 92.74% for TARGET 1 and 0 respectively. 

Both models generated AUC ROC of around 0.50, which again indicates that the models were not able to generate further class separation of the potential defaulting and non-defaulting customers.  This is due to a high skew of dataset towards non-defaulting customers.


### 3.4.4 Model Optimisation 2 - Bias reduction

In this optimisation, we will train the models using a balanced dataset of defaulting and non-defaulting customers.

### 3.4.4.1 Prepare unbiased training set

In [92]:
val df9 = train_df1.filter($"TARGET" === 1.0)
println("# of TARGET = 1: "+df9.count())

val df10 = train_df1.filter($"TARGET" === 0.0)
println("# of TARGET = 0: "+df10.count())

# of TARGET = 1: 9317
# of TARGET = 0: 119007


df9: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
df10: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [93]:
train_df1.printSchema

root
 |-- EXT_SOURCE_2: float (nullable = true)
 |-- EXT_SOURCE_3: float (nullable = true)
 |-- DAYS_BIRTH: float (nullable = true)
 |-- AMT_CREDIT: float (nullable = true)
 |-- AMT_ANNUITY: float (nullable = true)
 |-- DAYS_ID_PUBLISH: float (nullable = true)
 |-- previous_loans_CNT_PAYMENT_mean: float (nullable = true)
 |-- previous_loans_SELLERPLACE_AREA_max: float (nullable = true)
 |-- DAYS_LAST_PHONE_CHANGE: float (nullable = true)
 |-- DAYS_REGISTRATION: float (nullable = true)
 |-- CODE_GENDER_F: float (nullable = true)
 |-- REGION_POPULATION_RELATIVE: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_max_mean: float (nullable = true)
 |-- bureau_DAYS_CREDIT_mean: float (nullable = true)
 |-- previous_loans_NAME_CONTRACT_STATUS_Refused_count_norm: float (nullable = true)
 |-- client_installments_NUM_INSTALMENT_VERSION_max_mean: float (nullable = true)
 |-- bureau_DAYS_CREDIT_min: float (nullable = true)
 |-- previous_loans_NAME_YIELD_GROUP_high_count_norm: float (nullable

In [94]:
//Now from 119007 rows of TARGET==0, we randomly extract c.9317 rows and combine it with df9
//To get a dataframe of c.50% target==1 and c.50% target ==0

val df11 = df10.sample(false, 9222.toFloat/119415.toFloat)
df11.count()

df11: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]
res39: Long = 9174


In [95]:
val df12 = df9.union(df11)

df12: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [96]:
df12.cache

res40: df12.type = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 186 more fields]


In [97]:
// making sure that "features" does not include "SK_ID_CURR", "TARGET" or "label"

features.filter(x => x == "SK_ID_CURR" | x == "TARGET" | x == "label")

res41: Array[String] = Array()


### 3.4.4.2 Binomial Logistic Regression

In [98]:
val lrModel_unbiased = lrPipeline.fit(df12)

lrModel_unbiased: org.apache.spark.ml.PipelineModel = pipeline_092f8c8273b1


In [99]:
val testLr_unbiased = lrModel_unbiased.transform(validation_df1)

val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

val accuracy = evaluator.evaluate(testLr_unbiased)
println("Test set accuracy = " + accuracy)

val predictionLabelsRDD = testLr_unbiased.select(
    "prediction", "label").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics = new BinaryClassificationMetrics(predictionLabelsRDD)

println("Area under the receiver operating characteristic (ROC) curve : " + binMetrics.areaUnderROC)

Test set accuracy = 0.6951039790404454
Area under the receiver operating characteristic (ROC) curve : 0.690885742220752


testLr_unbiased: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 191 more fields]
evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_b41ce57d6eda
accuracy: Double = 0.6951039790404454
predictionLabelsRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1805] at map at <console>:73
binMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@34555342


### 3.4.4.3 Random Forest

In [101]:
val df13 = rfAssembler.transform(df12)

// Train model. This also runs the indexers.
val rfModel_unbiased = rfPipeline.fit(df13)

df13: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 187 more fields]
rfModel_unbiased: org.apache.spark.ml.PipelineModel = pipeline_f17db21b15b0


In [102]:
val df14 = rfAssembler.transform(validation_df1)

val testRf_unbiased = rfModel_unbiased.transform(df14)

// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(testRf_unbiased)
println("Test set accuracy = " + accuracy)

val predictionLabelsRDD = testRf_unbiased.select("prediction", "indexedLabel").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics = new BinaryClassificationMetrics(predictionLabelsRDD)

println("Area under the receiver operating characteristic (ROC) curve : " + binMetrics.areaUnderROC)

Test set accuracy = 0.6710878227171005
Area under the receiver operating characteristic (ROC) curve : 0.6547396510697723


df14: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 187 more fields]
testRf_unbiased: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 193 more fields]
evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_e0921f6ed999
accuracy: Double = 0.6710878227171005
predictionLabelsRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[1899] at map at <console>:75
binMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@4cd63015


### 3.4.4.4 Multilayer Perceptron Classifier Model

In [103]:
var dfTrain_unbiased = df12.withColumnRenamed("TARGET", "label")
dfTrain_unbiased = dfTrain_unbiased.drop("SK_ID_CURR")

var mlpcModel_unbiased = mlpcPipeline.fit(dfTrain_unbiased)

dfTrain_unbiased: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]
dfTrain_unbiased: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]
mlpcModel_unbiased: org.apache.spark.ml.PipelineModel = pipeline_c31d42af4b3a


In [104]:
var dfTest_unbiased = validation_df1.withColumnRenamed("TARGET", "label")
dfTest_unbiased = dfTest_unbiased.drop("SK_ID_CURR")

var testMlpc_unbiased = mlpcModel_unbiased.transform(dfTest_unbiased)

val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

val accuracy = evaluator.evaluate(testMlpc_unbiased)
println("Test set accuracy = " + accuracy)

val predictionLabelsRDD = testMlpc_unbiased.select("prediction", "indexedLabel").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics = new BinaryClassificationMetrics(predictionLabelsRDD)

println("Area under the receiver operating characteristic (ROC) curve : " + binMetrics.areaUnderROC)

Test set accuracy = 0.5241526117570001
Area under the receiver operating characteristic (ROC) curve : 0.5059246854449672


dfTest_unbiased: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]
dfTest_unbiased: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]
testMlpc_unbiased: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 191 more fields]
evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_cc0ff9419847
accuracy: Double = 0.5241526117570001
predictionLabelsRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[2244] at map at <console>:74
binMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@30778e0f


### 3.4.4.5 Model Optimisation 2 - conclusions

**Binomial Logistic Regression**
- Accuracy: 69.51%
- AUC ROC: 0.69

**Random Forest**
- Accuracy: 67.11%
- AUC ROC: 0.65

**Multilayer Perceptron Classifier**
- Accuracy: 52.42%
- AUC ROC: 0.51

There were significant improvements to the metric AUC ROC for the logistic regression and random forest models. The models are now providing class separation of defaulting and non-defaulting customers.

The optmisation has resulted in decreases in accuracy.  The appropriateness of trade-off of accuracy and AUC ROC in this optimisation would be determined with business considerations. The higher AUC ROC indicates we can predict potential defaults better but it leads to more false positives.  Would the reduced risk of taking on potential defaults compensate for the profits forfeited from turning down new loans?

### 3.5 Model Testing

From the model fitting and optimisation, the best performing models are the ones fitted with unbiased datasets.
- lrModel_unbiased
- rfModel_unbiased
- mlpcModel_unbiased

These have been selected for testing with the hidden test dataset - test_df1.

### 3.5.1 Logistic Regression Model

In [105]:
val testLr = lrModel_unbiased.transform(test_df1)

val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

val accuracy = evaluator.evaluate(testLr)
println("Test set accuracy = " + accuracy)

val predictionLabelsRDD = testLr.select("prediction", "label").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics = new BinaryClassificationMetrics(predictionLabelsRDD)

println("Area under the receiver operating characteristic (ROC) curve : " + binMetrics.areaUnderROC)

Test set accuracy = 0.692344883158743
Area under the receiver operating characteristic (ROC) curve : 0.6889831920968008


testLr: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 191 more fields]
evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_7f619c3aba97
accuracy: Double = 0.692344883158743
predictionLabelsRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[2290] at map at <console>:71
binMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@55cb7833


### 3.5.2 Random Forest Model

In [106]:
val df5 = rfAssembler.transform(test_df1)

val testRf = rfModel_unbiased.transform(df4)

// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(testRf)
println("Test set accuracy = " + accuracy)

val predictionLabelsRDD = testRf.select("prediction", "indexedLabel").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics = new BinaryClassificationMetrics(predictionLabelsRDD)

println("Area under the receiver operating characteristic (ROC) curve : " + binMetrics.areaUnderROC)

Test set accuracy = 0.6846689895470384
Area under the receiver operating characteristic (ROC) curve : 0.6531592643105513


df5: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 187 more fields]
testRf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 193 more fields]
evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_227784265861
accuracy: Double = 0.6846689895470384
predictionLabelsRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[2326] at map at <console>:77
binMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@4b42dcf5


### 3.5.3 Multilayer Perceptron Classifier Model

In [107]:
var dfTest = test_df1.withColumnRenamed("TARGET", "label")
dfTest = dfTest.drop("SK_ID_CURR")

var testMlpc = mlpcModel_unbiased.transform(dfTest)

val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

val accuracy = evaluator.evaluate(testMlpc)
println("Test set accuracy = " + accuracy)

val predictionLabelsRDD = testMlpc.select("prediction", "indexedLabel").rdd.map(r => (r.getDouble(0), r.getDouble(1)))
val binMetrics = new BinaryClassificationMetrics(predictionLabelsRDD)

println("Area under the receiver operating characteristic (ROC) curve : " + binMetrics.areaUnderROC)

Test set accuracy = 0.5170024174053183
Area under the receiver operating characteristic (ROC) curve : 0.509329619065342


dfTest: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]
dfTest: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 185 more fields]
testMlpc: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: float, EXT_SOURCE_3: float ... 191 more fields]
evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_4cb4762162fc
accuracy: Double = 0.5170024174053183
predictionLabelsRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[2369] at map at <console>:76
binMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@78eaf4bd


### 3.5.4 Model testing conclusions

**Binomial Logistic Regression**
- Accuracy: 69.23%
- AUC ROC: 0.69

**Random Forest**
- Accuracy: 68.47%
- AUC ROC: 0.65

**Multilayer Perceptron Classifier**
- Accuracy: 51.70%
- AUC ROC: 0.51

Testing results are similar to those in validation.  The Binomial Logistic Regression provides the best results for both the Accuracy and AUC ROC metrics.  This is followed by Random Forest with comparable results and then the Multilayer Perceptron Classifier.